In [0]:
from google.colab import drive
from glob import glob
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import csv
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.utils.data as Data
from matplotlib import cm

In [2]:
drive.mount("/content/drive")
path = 'drive/My Drive/faces94' 
files = glob(f"{path}/**/**/*.jpg")
c2i = {
    "female": 0,
    "male":1,
    "malestaff": 2 
}

c_files = files #[:50]
import torchvision.transforms as transforms

transform_loader = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
total_images = torch.stack([transform_loader(Image.open(x)) for x in c_files])
total_labels = torch.tensor([c2i[x.split('/')[-3]] for x in c_files])

index_train, index_test = train_test_split(range(len(total_images)), test_size=0.2, random_state=102)

train_image = total_images[index_train]
train_label = total_labels[index_train]

test_image = total_images[index_test]
test_label = total_labels[index_test]

noOfEpochs = 10
size_of_batch = 64
eeta = 0.005         
dataset_download = True
test_images_count = 5

In [0]:
class autoEncoder(nn.Module):
    def __init__(self):
        super(autoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(3*200*180,2000 ),
            nn.Tanh(),
            nn.Linear(2000, 1000),
            nn.Tanh(),
            nn.Linear(1000, 200),
            nn.Tanh(),
            nn.Linear(200, 100),   # compress to 3 features which can be visualized in plt
        )
        self.decoder = nn.Sequential(
            nn.Linear(100, 200),
            nn.Tanh(),
            nn.Linear(200, 1000),
            nn.Tanh(),
            nn.Linear(1000, 2000),
            nn.Tanh(),
            nn.Linear(2000, 3*200*180),
            nn.Sigmoid(),       # compress to a range (0, 1)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [0]:
autoEncoder = autoEncoder()

optimizer = torch.optim.Adam(autoEncoder.parameters(), lr=eeta)
loss_function = nn.MSELoss()

output_file = open('output.csv', 'w')
outputList = []
print(train_label[0])
for noOfEpochs in range(noOfEpochs):
    for step,x in enumerate(train_image):
        b_x = x.view(-1, 3*200*180)   # batch x, shape (batch, 28*28)
        b_y = x.view(-1, 3*200*180)   # batch y, shape (batch, 28*28)
        encoded, decoded = autoEncoder(b_x)
        li = []
        
        li = encoded.detach().numpy().tolist();
        li = li[0]
        print(li)
        outputList.append(li); #appending encoded image
        outputList.append(train_label[step])
        loss = loss_function(decoded, b_y)      # mean square error
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients

        if step % 100 == 0:
            print('noOfEpochs: ', noOfEpochs, '| train loss: %.4f' % loss.data.numpy())

tensor(1)
[0.04861484467983246, -0.0669759064912796, 0.03922516852617264, 0.05992425978183746, 0.17433378100395203, -0.06788696348667145, 0.08884594589471817, -0.1075010746717453, 0.02321121096611023, 0.06106927618384361, 0.02039877511560917, 0.05212844908237457, -0.07981979101896286, -0.015615101903676987, -0.08645591884851456, -0.0011903494596481323, 0.005055472254753113, 0.02937234193086624, 0.040755800902843475, 0.0618453212082386, 0.007154615595936775, -0.047085341066122055, 0.031027331948280334, -0.06998149305582047, 0.07431747764348984, -0.07101485133171082, -0.09123602509498596, 0.01496167853474617, -0.04091088846325874, 0.03091440349817276, 0.013181682676076889, -0.05238592252135277, 0.17508038878440857, -0.0695321336388588, -0.045423295348882675, 0.02126821130514145, -0.06020013615489006, -0.013040106743574142, 0.017989035695791245, -0.0583009198307991, -0.10448411107063293, -0.1308140605688095, -0.07537677139043808, 0.030501432716846466, 0.1321784257888794, 0.096973404288291

In [0]:
with output_file:
    writer = csv.writer(output_file)
    writer.writerows(outputList)


for step,x in enumerate(noOfEpochs):
    b_x = x.view(-1, 3*200*180)   # batch x, shape (batch, 28*28)
    b_y = x.view(-1, 3*200*180)   # batch y, shape (batch, 28*28)
    #print(b_x)
    encoded, decoded = autoEncoder(b_x)

    loss = loss_function(decoded, b_y)      # mean square error

    if step % 100 == 0:
        print('noOfEpochs: ', noOfEpochs, '| test loss: %.4f' % loss.data.numpy())